In [1]:
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from numpy import array
import tensorflow as tf
from random import shuffle

TRAIN_DIR = 'test_train_image/all/7to3/mfcc_14/train'
train_folder_list = array(os.listdir(TRAIN_DIR))
#print(train_folder_list)

trains = []
tests = []

train_input = []
train_label = []

label_encoder = LabelEncoder()  # LabelEncoder Class 호출
integer_encoded = label_encoder.fit_transform(train_folder_list)
onehot_encoder = OneHotEncoder(sparse=False)
#print(len(integer_encoded))
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded) #여기서 에러뜸
#print(onehot_encoded)

for index in range(len(train_folder_list)):
    path = os.path.join(TRAIN_DIR, train_folder_list[index])
    path = path + '/'
    img_list = os.listdir(path)
    for img in img_list:
        img_path = os.path.join(path, img)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        train_input.append([np.array(img)])
        train_label.append([np.array(onehot_encoded[index])])
        #trains.append(([np.array(img)],[np.array(onehot_encoded[index])]))
        
#random.shuffle(trains)
train_tmp = [[x,y] for x,y in zip(train_input,train_label)]
shuffle(train_tmp)
train_input=[arr[0] for arr in train_tmp]
train_label=[arr[1] for arr in train_tmp]

'''
for (i,j) in trains:
    train_input.append(i)
    train_label.append(j)
'''
train_input = np.reshape(train_input, (-1, 196))
train_label = np.reshape(train_label, (-1, 8))
train_input = np.array(train_input).astype(np.float32)
train_label = np.array(train_label).astype(np.float32)

#print(train_input)
print(train_label)

np.save("train_data.npy", train_input)
np.save("train_label.npy", train_label)

'''test data 설계'''

TEST_DIR = 'test_train_image/all/7to3/mfcc_14/test'
test_folder_list = array(os.listdir(TEST_DIR))
 
test_input = []
test_label = []
 
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(test_folder_list)
 
onehot_encoder = OneHotEncoder(sparse=False) 
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
 
for index in range(len(test_folder_list)):
    path = os.path.join(TEST_DIR, test_folder_list[index])
    path = path + '/'
    img_list = os.listdir(path)
    for img in img_list:
        img_path = os.path.join(path, img)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        test_input.append([np.array(img)])
        test_label.append([np.array(onehot_encoded[index])])
        #tests.append(([np.array(img)],[np.array(onehot_encoded[index])]))

#random.shuffle(tests)
tests_tmp = [[x,y] for x,y in zip(test_input,test_label)]
shuffle(tests_tmp)
test_input=[arr[0] for arr in tests_tmp]
test_label=[arr[1] for arr in tests_tmp]
'''
for (i,j) in tests:
    test_input.append(i)
    test_label.append(j)
'''
test_input = np.reshape(test_input, (-1, 196))
test_label = np.reshape(test_label, (-1, 8))
test_input = np.array(test_input).astype(np.float32)
test_label = np.array(test_label).astype(np.float32)

np.save("test_input.npy",test_input)
np.save("test_label.npy",test_label)

#print(test_input)
print(test_label)


c:\users\dong\appdata\local\programs\python\python37\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


c:\users\dong\appdata\local\programs\python\python37\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [2]:
# hyper parameters
learning_rate = 0.001
 
# input place holders
X = tf.placeholder(tf.float32, [None, 196])
X_img = tf.reshape(X, [-1, 14, 14, 1])   # img 28x28x1 (black/white) #img 356x238x1
Y = tf.placeholder(tf.float32, [None, 8])

W1 = tf.Variable(tf.random_normal([3,3,1,64],stddev =0.01))
L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)
tf.layers.batch_normalization(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

 
W2 = tf.Variable(tf.random_normal([3, 3, 64, 128], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
tf.layers.batch_normalization(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
 
W3 = tf.Variable(tf.random_normal([3, 3, 128, 256], stddev=0.01))
L3 = tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding='SAME')
L3 = tf.nn.relu(L3)
tf.layers.batch_normalization(L3)
L3 = tf.nn.max_pool(L3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
 
L3_flat = tf.reshape(L3, [-1, 1024])
W4 = tf.get_variable("W4", shape=[1024, 8], initializer=tf.contrib.layers.xavier_initializer())
b = tf.Variable(tf.random_normal([8]))
logits = tf.matmul(L3_flat, W4) + b
 
# define cost/loss &amp;amp;amp; optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.batch_normalization instead.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [3]:
training_epochs = 15
batch_size = 200
 
# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())
 
# train my model
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(len(train_input) / batch_size)
 
    for i in range(total_batch):
        start = ((i + 1) * batch_size) - batch_size
        end = ((i + 1) * batch_size)
        batch_xs = train_input[start:end]
        batch_ys = train_label[start:end]
        feed_dict = {X: batch_xs, Y: batch_ys}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch
 
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))
 
print('Learning Finished!')
 
# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: test_input, Y: test_label}))

Learning started. It takes sometime.
Epoch: 0001 cost = 1.720761021
Epoch: 0002 cost = 0.732486387
Epoch: 0003 cost = 0.675039192
Epoch: 0004 cost = 0.661178350
Epoch: 0005 cost = 0.652180533
Epoch: 0006 cost = 0.648144484
Epoch: 0007 cost = 0.648819685
Epoch: 0008 cost = 0.645659288
Epoch: 0009 cost = 0.646367530
Epoch: 0010 cost = 0.643389622
Epoch: 0011 cost = 0.643650790
Epoch: 0012 cost = 0.642251968
Epoch: 0013 cost = 0.641389072
Epoch: 0014 cost = 0.641350349
Epoch: 0015 cost = 0.640563607
Learning Finished!
Accuracy: 0.87636364
